In [1]:
import os
import urllib
import shutil
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

import collections
from sklearn.model_selection import train_test_split
import pickle as pkl
import logging
import azureml.dataprep as dprep 
from azureml.core import Dataset

In [2]:
data_df = pd.read_csv('data/ML_Training_data_Risk_Sales_v1.csv', low_memory=False)
test_df= pd.read_csv('data/ML_Testing_data_Risk_Sales_v1.csv', low_memory=False)

In [3]:
data_df.isnull().values.any()
data_df.isnull().sum()

SALES_CHANNEL                       0
TOTAL_PRICE                         0
MAKE1                               8
MODEL1                              8
PRICE1                             36
MAKE2                           76167
MODEL2                          76167
PRICE2                          76168
MAKE3                           92522
MODEL3                          92522
PRICE3                          92522
MAKE4                           95194
MODEL4                          95194
PRICE4                          95194
MAKE5                           96169
MODEL5                          96169
PRICE5                          96169
NUM_PORTIN                          0
ACNT_BILL_FNAME_MATCHES             0
ACNT_BILL_LNAME_MATCHES             0
ACNT_SHIP_FNAME_MATCHES             0
ACNT_SHIP_LNAME_MATCHES             0
SHIP_BILL_FNAME_MATCHES             0
SHIP_BILL_LNAME_MATCHES             0
BILL_SHIP_ADDR_MATCHES              0
NUM_BYOD                            0
IS_EXISTING_

AttributeError: 'DataFrame' object has no attribute 'series'

In [7]:
data_df.fillna(data_df.median(), inplace=True)
test_df.fillna(data_df.median(), inplace=True)

In [8]:
data_df.head()

,SALES_CHANNEL,TOTAL_PRICE,MAKE1,MODEL1,PRICE1,MAKE2,MODEL2,PRICE2,MAKE3,MODEL3,...,FRAUDSTATUSIND,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,Risk Label,START_TO_CLOSE_MINS
0,ONLINE,631.56,Samsung,Galaxy Note8,599.99,NaN,NaN,NaN,NaN,NaN,...,Green,19.50,10.77,233555.88,3834.27,3825.98,3789.93,0,Good,2421.68
1,TELESALES,126.96,Motorola,moto e5 play,119.99,NaN,NaN,NaN,NaN,NaN,...,Green,19.50,22.35,19.82,3834.27,3825.98,3789.93,0,Good,1634.05
2,TELESALES,1401.76,Apple,iPhone XS Max,1149.99,Case-Mate,Apple iPhone XS Max Case & GLASS Screen Protec...,44.95,NaN,NaN,...,Red,19.30,22.05,305.78,4330.95,2988.13,4292.47,1,Fraud,1342.82
3,TELESALES,0.00,Apple,iPhone 6s Plus,0,NaN,NaN,NaN,NaN,NaN,...,Green,19.50,19.05,9.83,3834.27,3825.98,3789.93,0,Good,1860.17
4,ONLINE,1337.42,Apple,iPhone XS Max,1149.99,NaN,NaN,NaN,NaN,NaN,...,Red,17.93,20.18,1025688.28,3650.92,3825.98,3626.18,1,Fraud,6603.47


In [9]:
data=data_df[['TOTAL_PRICE','FINAL_RESULT','HOUR_OF_DAY','ACTIVE_WATCH_DURATION_MINS','ORDER_HOLD_DURATION_MINS','ACCOUNT_OPEN_TO_ORDER_MINS','CASECLOSE_TO_ORDERCLOSE_MINS','CASEOPEN_TO_CASECLOSE_MINS','Risk Label','START_TO_CLOSE_MINS']]
test_data=test_df[['TOTAL_PRICE','FINAL_RESULT','HOUR_OF_DAY','ACTIVE_WATCH_DURATION_MINS','ORDER_HOLD_DURATION_MINS','ACCOUNT_OPEN_TO_ORDER_MINS','CASECLOSE_TO_ORDERCLOSE_MINS','CASEOPEN_TO_CASECLOSE_MINS','Risk Label','START_TO_CLOSE_MINS']]

In [10]:
data.shape

(96458, 10)

In [11]:
test_data.shape

(36997, 10)

In [12]:
data.isnull().values.any()
data.isnull().sum()

TOTAL_PRICE                     0
FINAL_RESULT                    0
HOUR_OF_DAY                     0
ACTIVE_WATCH_DURATION_MINS      0
ORDER_HOLD_DURATION_MINS        0
ACCOUNT_OPEN_TO_ORDER_MINS      0
CASECLOSE_TO_ORDERCLOSE_MINS    0
CASEOPEN_TO_CASECLOSE_MINS      0
Risk Label                      0
START_TO_CLOSE_MINS             0
dtype: int64

In [13]:
test_data.isnull().sum()

TOTAL_PRICE                     0
FINAL_RESULT                    0
HOUR_OF_DAY                     0
ACTIVE_WATCH_DURATION_MINS      0
ORDER_HOLD_DURATION_MINS        0
ACCOUNT_OPEN_TO_ORDER_MINS      0
CASECLOSE_TO_ORDERCLOSE_MINS    0
CASEOPEN_TO_CASECLOSE_MINS      0
Risk Label                      0
START_TO_CLOSE_MINS             0
dtype: int64

In [34]:
#test_data['ACTIVE_WATCH_DURATION_MINS'].fillna('NA',inplace=True)
#test_data['CASECLOSE_TO_ORDERCLOSE_MINS'].fillna('NA',inplace=True)
#test_data['CASEOPEN_TO_CASECLOSE_MINS'].fillna('NA',inplace=True)
#test_data['START_TO_CLOSE_MINS'].fillna('NA',inplace=True)

In [14]:
test_data.head()

,TOTAL_PRICE,FINAL_RESULT,HOUR_OF_DAY,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,Risk Label,START_TO_CLOSE_MINS
0,847.92,YELLOW,16,18.65,21.18,7.83,3825.98,2820.68,Fraud,6603.47
1,0.00,GREEN,23,20.32,93.03,8.30,3825.98,3789.93,Good,11056.48
2,126.96,GREEN,1,19.50,22.65,1.77,3825.98,3789.93,Good,8048.73
3,655.26,GREEN,19,19.50,10.97,1467.63,3825.98,3789.93,Good,7060.18
4,479.57,GREEN,11,18.95,43.45,59000.28,3825.98,3789.93,Good,4889.35


In [15]:
def estimated(row):
    duration = row['START_TO_CLOSE_MINS']
#     print(color)
    if duration <= 1440 :
        return 0
    elif duration >1441 and duration <=4320 :
        return 1
    elif duration >4321 and duration <=7200 :
        return 2
    elif duration >7201 and duration <=11520 :
        return 3
    else:
        return 4

In [16]:
data['label'] = data.apply(estimated, axis=1)
data.drop(columns=['START_TO_CLOSE_MINS'], inplace=True)

test_data['label'] = test_data.apply(estimated, axis=1)
test_data.drop(columns=['START_TO_CLOSE_MINS'], inplace=True)

In [17]:
data.head()

,TOTAL_PRICE,FINAL_RESULT,HOUR_OF_DAY,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,Risk Label,label
0,631.56,GREEN,0,19.50,10.77,233555.88,3825.98,3789.93,Good,1
1,126.96,GREEN,19,19.50,22.35,19.82,3825.98,3789.93,Good,1
2,1401.76,RED,23,19.30,22.05,305.78,2988.13,4292.47,Fraud,0
3,0.00,RED,14,19.50,19.05,9.83,3825.98,3789.93,Good,1
4,1337.42,RED,12,17.93,20.18,1025688.28,3825.98,3626.18,Fraud,2


In [18]:
test_data['label'].max()

4

In [19]:
collections.Counter(data['label'])

Counter({1: 25741, 0: 2273, 2: 29566, 4: 7637, 3: 31241})

In [20]:
oneday_df = data[data['label'] == 0]
threeday_df = data[data['label'] == 1]
fiveday_df = data[data['label'] == 2]
eightday_df = data[data['label'] == 3]
stallday_df = data[data['label'] == 4]

In [21]:
oneday_over_df = oneday_df.sample(25741, replace=True)
fiveday_df_over_df = fiveday_df.sample(25741, replace=True)
eightday_df_under_df = eightday_df.sample(25741, replace=True)
stallday_df_over_df = stallday_df.sample(25741, replace=True)

In [22]:
balanced_data_df = pd.concat([oneday_over_df,fiveday_df_over_df,eightday_df_under_df,stallday_df_over_df])

In [23]:
balanced_data_df.shape

(102964, 10)

In [24]:
X_train = balanced_data_df.drop(columns=['label'])
Y_train = balanced_data_df['label']

X_test = test_data.drop(columns=['label'])
Y_test = test_data['label']

In [25]:
X_train.shape

(102964, 9)

In [26]:
Y_train = np.array(Y_train)

In [27]:
Y_train.shape

(102964,)

In [28]:
X_test.shape

(36997, 9)

In [29]:
Y_test.shape

(36997,)

In [30]:
balanced_data_df.to_csv('data/EstimatedShipping_train.csv',index=False)
test_data.to_csv('data/EstimatedShipping_test.csv',index=False)

In [31]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'EstimatedShipping'
# project folder
project_folder = './ml_experiment/EstimatedShipping'
 
experiment = Experiment(ws, experiment_name)
 
output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.69
Subscription ID,be564fde-136b-4709-b7b6-abfc0bdfc134
Workspace Name,xm-ml-workspace
Resource Group,xm-ml
Location,centralus
Project Directory,./ml_experiment/EstimatedShipping
Experiment Name,EstimatedShipping


In [32]:
train_dflow = dprep.read_csv('data/EstimatedShipping_train.csv', infer_column_types=True)
train_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
TOTAL_PRICE,FieldType.DECIMAL,0.00,7992.25,102964.00,0.00,102964.00,0.00,0.00,0.00,>1000,0.00,0.00,0.00,0.00,263.73,837.58,1909.82,3117.74,5372.37,545.72,711.52,506259.55,2.29,9.21,2056.58,0.00
FINAL_RESULT,FieldType.STRING,GREEN,YELLOW,102964.00,0.00,102964.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
HOUR_OF_DAY,FieldType.INTEGER,0.00,23.00,102964.00,0.00,102964.00,0.00,0.00,0.00,24,0.00,1.00,1.00,13.24,17.00,20.00,23.00,23.00,23.00,15.07,7.13,50.79,-1.07,-0.15,23.00,3.97
ACTIVE_WATCH_DURATION_MINS,FieldType.DECIMAL,9.45,74.68,102964.00,0.00,102964.00,0.00,0.00,0.00,931,13.20,16.74,16.64,19.50,19.50,19.50,23.43,25.32,28.02,19.50,2.14,4.56,0.73,15.24,19.50,19.50
ORDER_HOLD_DURATION_MINS,FieldType.DECIMAL,1.30,9997.22,102964.00,0.00,102964.00,0.00,0.00,0.00,>1000,5.35,11.16,11.07,15.21,19.05,25.51,154.28,267.27,784.83,38.06,68.79,4731.82,34.01,4310.85,38.62,1.30
ACCOUNT_OPEN_TO_ORDER_MINS,FieldType.DECIMAL,0.03,1447595.30,102964.00,0.00,102964.00,0.00,0.00,0.00,>1000,0.24,2.53,2.44,8.07,3904.46,140013.35,819157.19,1036714.57,1172006.84,137532.80,259818.06,67505422907.30,2.14,3.69,348294.99,0.03
CASECLOSE_TO_ORDERCLOSE_MINS,FieldType.DECIMAL,0.95,162474.42,102964.00,0.00,102964.00,0.00,0.00,0.00,>1000,99.05,3825.98,3825.98,3825.98,3825.98,3825.98,6035.27,11165.67,30204.73,4096.41,2237.93,5008314.41,20.31,994.96,3825.98,3825.98
CASEOPEN_TO_CASECLOSE_MINS,FieldType.DECIMAL,20.37,6524.43,102964.00,0.00,102964.00,0.00,0.00,0.00,>1000,832.52,3504.45,3454.99,3789.93,3789.93,3789.93,5277.72,6480.03,6480.07,3818.48,712.93,508265.94,0.99,7.07,3789.93,3789.93
Risk Label,FieldType.STRING,Collections,Good,102964.00,0.00,102964.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
label,FieldType.INTEGER,0.00,4.00,102964.00,0.00,102964.00,0.00,0.00,0.00,4,0.00,0.00,0.00,0.99,2.39,3.52,4.00,4.00,4.00,2.25,1.48,2.19,-0.43,-1.15,4.00,0.00


In [33]:
test_dflow = dprep.read_csv('data/EstimatedShipping_test.csv', infer_column_types=True)
test_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
TOTAL_PRICE,FieldType.DECIMAL,0.00,6890.15,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,0.00,0.00,0.00,0.00,378.50,886.31,1642.45,2668.87,5043.41,559.61,644.53,415419.57,1.97,8.05,2153.59,0.00
FINAL_RESULT,FieldType.STRING,GREEN,YELLOW,36997.00,0.00,36997.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
HOUR_OF_DAY,FieldType.INTEGER,0.00,23.00,36997.00,0.00,36997.00,0.00,0.00,0.00,24,0.00,1.60,1.00,13.98,17.00,20.00,23.00,23.00,23.00,15.16,6.94,48.14,-1.11,0.01,23.00,5.79
ACTIVE_WATCH_DURATION_MINS,FieldType.DECIMAL,7.50,29.92,36997.00,0.00,36997.00,0.00,0.00,0.00,868,14.73,19.50,19.50,19.50,19.50,23.08,27.08,28.54,29.44,21.24,2.82,7.93,1.02,0.11,28.41,15.72
ORDER_HOLD_DURATION_MINS,FieldType.DECIMAL,1.42,1288.20,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,10.19,14.64,14.48,18.52,24.11,32.04,158.83,290.71,389.75,42.97,54.30,2948.91,3.56,19.84,49.12,1.42
ACCOUNT_OPEN_TO_ORDER_MINS,FieldType.DECIMAL,0.03,1549937.45,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,0.42,2.52,2.46,6.35,1283.73,121671.44,881253.15,1106461.08,1230996.89,143003.60,279419.90,78075477987.57,2.16,3.69,301931.19,0.03
CASECLOSE_TO_ORDERCLOSE_MINS,FieldType.DECIMAL,110.35,82270.10,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,1398.09,3825.98,3825.98,3825.98,3825.98,3825.98,4031.77,9065.44,18388.02,3984.71,1369.79,1876327.41,22.57,993.23,3825.98,3825.98
CASEOPEN_TO_CASECLOSE_MINS,FieldType.DECIMAL,1444.33,6482.22,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,1490.65,3789.93,3789.93,3789.93,3789.93,3789.93,4279.38,6480.03,6480.07,3830.07,609.46,371436.41,2.15,11.66,3789.93,3789.93
Risk Label,FieldType.STRING,Collections,Good,36997.00,0.00,36997.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
label,FieldType.INTEGER,0.00,4.00,36997.00,0.00,36997.00,0.00,0.00,0.00,5,0.00,1.00,1.00,2.00,3.00,3.00,4.00,4.00,4.00,2.45,0.86,0.75,-0.50,0.03,4.00,0.94


In [34]:
X_df = train_dflow.drop_columns(columns=['label'])
y_df = train_dflow.keep_columns(columns=['label'], validate_column_exists=True)
X_test_df = test_dflow.drop_columns(columns=['label'])
y_test_df = test_dflow.keep_columns(columns=['label'], validate_column_exists=True)

In [35]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

cts = ws.compute_targets
amlcompute_cluster_name = "XM-Heavy-Load"
compute_target = cts[amlcompute_cluster_name]



found = False

# # Check if this compute target already exists in the workspace.

cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                vm_priority = 'lowpriority',
                                                                min_nodes = 0, max_nodes = 6)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

#     # Can poll for a minimum number of nodes and for a specific timeout.
#     # If no min_node_count is provided, it will use the scale settings for the cluster.
#     compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

#      # For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.


In [36]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

# dprep_dependency = 'azureml-dataprep==' + pkg_resources.get_distribution("azureml-dataprep").version

cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', 'psutil'], conda_packages=['numpy','py-xgboost<=0.80'])
# cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', dprep_dependency], conda_packages=['numpy','py-xgboost<=0.80'])
conda_run_config.environment.python.conda_dependencies = cd

In [37]:
# Dump X and Y to CSVs.
X_df.to_pandas_dataframe().to_csv('data/shipment/X_train.csv', index=False)
y_df.to_pandas_dataframe().to_csv('data/shipment/Y_train.csv', index=False)

# Dump X and Y to CSVs.
X_test_df.to_pandas_dataframe().to_csv('data/shipment/X_test.csv', index=False)
y_test_df.to_pandas_dataframe().to_csv('data/shipment/Y_test.csv', index=False)

In [38]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./data/shipment', target_path='data/shipment', overwrite=True, show_progress=True)

Uploading an estimated of 4 files
Uploading ./data/shipment/X_test.csv
Uploading ./data/shipment/X_train.csv
Uploading ./data/shipment/Y_test.csv
Uploading ./data/shipment/Y_train.csv
Uploaded ./data/shipment/Y_train.csv, 1 files out of an estimated total of 4
Uploaded ./data/shipment/Y_test.csv, 2 files out of an estimated total of 4
Uploaded ./data/shipment/X_test.csv, 3 files out of an estimated total of 4
Uploaded ./data/shipment/X_train.csv, 4 files out of an estimated total of 4
Uploaded 4 files


$AZUREML_DATAREFERENCE_8aaf2747de1548d89635abfc3ccf4cb1

In [39]:
X = Dataset.Tabular.from_delimited_files(path=ds.path('data/shipment/X_train.csv'))
y = Dataset.Tabular.from_delimited_files(path=ds.path('data/shipment/Y_train.csv'))

X_valid = Dataset.Tabular.from_delimited_files(path=ds.path('data/shipment/X_test.csv'))
y_valid = Dataset.Tabular.from_delimited_files(path=ds.path('data/shipment/Y_test.csv'))

In [40]:
automl_config = AutoMLConfig(task = 'classification',
                             primary_metric = 'AUC_weighted',
                             debug_log='automl_errors.log',
                             path = project_folder, #For external compute
                             run_configuration=conda_run_config, #For external compute
                             preprocess=True,
                             X = X, 
                             y = y,
                             X_valid=X_valid,
                             y_valid=y_valid,
                             iteration_timeout_minutes=10,
                             experiment_timeout_minutes=200,
                             max_concurrent_iterations=10,
                             max_cores_per_iteration=4,
                             iterations=100,
                             model_explainability=True,
                             )

In [ ]:
# New Lines
# conda_env.docker.enabled = True
# conda_env.python.user_managed_dependencies = False

# print('Base Docker image is:', conda_env.docker.base_image)
# END  New Lines
ws = Workspace.from_config()

experiment_name = 'EstimatedShipping'

experiment = Experiment(ws, experiment_name)
#local_run = experiment.submit(automl_config, show_output = True)
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote or ADB.
Running on remote compute: XM-Heavy-Load
Parent Run ID: AutoML_f4bcdde1-36e5-4c50-a235-1822086e27ce
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         1   MaxAbsScaler SGD                               0:01:30       0.5934    0.5934
         6   MaxA

        71   MaxAbsScaler GradientBoosting                  0:03:09       0.6239    0.6541
        79   MaxAbsScaler LightGBM                          0:01:42       0.6281    0.6541
        83   MaxAbsScaler LightGBM                          0:01:37       0.6455    0.6541
        81   MaxAbsScaler LogisticRegression                0:02:16       0.6080    0.6541
        85   StandardScalerWrapper LightGBM                 0:01:28       0.6515    0.6541
        84   MaxAbsScaler LightGBM                          0:01:37       0.6252    0.6541
        86   StandardScalerWrapper LightGBM                 0:01:30       0.6129    0.6541
        88   StandardScalerWrapper LightGBM                 0:01:34       0.6359    0.6541
        89   StandardScalerWrapper LightGBM                 0:01:37       0.6550    0.6550
        90   MaxAbsScaler LightGBM                          0:01:37       0.6254    0.6550
        93   StandardScalerWrapper LightGBM                 0:01:24       0.6432    0.6550

In [102]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
EstimatedShipping,AutoML_f96f2d47-4cd1-4333-b83b-630f466db848,automl,Completed,Link to Azure Portal,Link to Documentation


In [103]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: EstimatedShipping,
Id: AutoML_f96f2d47-4cd1-4333-b83b-630f466db848_99,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('stac...7f6471e4b7f0>,
           solver='lbfgs', tol=0.0001, verbose=0),
            training_cv_folds=5))])


In [104]:
print(overall_summary)
print(overall_imp)
print(per_class_summary)
print(per_class_imp)

[82581.17936681156, 28299.876667826702, 23003.752118361208, 3126.1658203002426, 2749.649595197979, 2258.1664907520785, 2003.4724845862154, 778.5107854115246, 500.4728878466681, 406.5297185422721, 199.26780028763383, 78.29088938427327, 29.454670048807024, 28.872182625867236, 20.801320179979857, 18.888787751743532, 13.709129271459258, 8.342154709661779, 6.093765385404226, 4.321086212691751, 4.012299941585305, 3.394367560749088, 2.8607555490374907, 2.500973917723351, 2.263839273729871, 1.9704602704528806, 1.2559231778689492, 0.8252659086091907, 0.6742364855422113, 0.46516778837147416, 0.12032639336661315, 0.1140854160124586, 0.09040289501261299, 0.04589089941674802, 0.036379934710778696, 0.03206845695312715, 0.023949832016430965, 0.016634767132499447, 0.0028239805799601314]
['ORDER_HOLD_DURATION_MINS_MeanImputer', 'ACTIVE_WATCH_DURATION_MINS_MeanImputer', 'TOTAL_PRICE_MeanImputer', 'HOUR_OF_DAY_CharGramCountVectorizer_22', 'FINAL_RESULT_CharGramCountVectorizer_red', 'HOUR_OF_DAY_CharGramC